In [ ]:
%pip install transformers datasets torch scikit-learn streamlit
%pip install tf-keras
%pip install pandas

In [ ]:
'../dataset/depression_emo.json'

In [4]:
import pandas as pd
import json

# Load the JSON file
with open('../dataset/depression_emo.json', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# Convert to DataFrame
df = pd.DataFrame(data)

# View the first few rows
df.head()


,id,title,post,text,upvotes,date,feelings
0,17cs3k1,Feeling Lonely Even When Not Alone,"I have people close to me willing to help, but...",Feeling Lonely Even When Not Alone ### I have ...,3,2023-10-21 02:20:08,"[loneliness, lack of energy, hopelessness, sad..."
1,16yr7wu,I'm failing my classes and I don't know what t...,I'm tired of it all and it tears me apart that...,I'm failing my classes and I don't know what t...,6,2023-10-03 12:56:51,"[lack of energy, hopelessness, self hate, sadn..."
2,yt332q,Does it ever go away?,The emptiness of losing someone you love? The ...,Does it ever go away? ### The emptiness of los...,314,2022-11-12 10:19:29,"[emptiness, sadness, hopelessness, worthlessne..."
3,17de9m5,Thank you to whoever made this subreddit,Hi. Just wanted to say I'm glad I found this s...,Thank you to whoever made this subreddit ### H...,6,2023-10-21 22:27:42,[loneliness]
4,zcpexp,I was suicidal yesterday and now I'm too emoti...,"Idk if this is the right subreddit for this, s...",I was suicidal yesterday and now I'm too emoti...,285,2022-12-04 23:24:01,"[lack of energy, suicide intent, emptiness, ho..."


Creating dataset for intents classification

In [ ]:
import pandas as pd
import json

# Load the JSON file
with open('../dataset/data_full.json', 'r') as f:
    data = json.load(f)
    
# Create DataFrame from the list of lists in 'oos_val'
df_train_raw = pd.DataFrame(data['train'], columns=['query', 'label'])

# Save the DataFrame to a CSV file

df_train_raw.to_csv('data_full.csv', index=False)
df_train_raw

JSONDecodeError: Extra data: line 2 column 1 (char 2258)

In [ ]:
# Path to your JSON file
input_file = 'paste.txt'
output_file = 'reddit_depress.csv'

# Read the file and process each line as a separate JSON object
data = []
with open(input_file, 'r', encoding='utf-8') as file:
    for line in file:
        if line.strip():  # Skip empty lines
            data.append(json.loads(line))

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert the feelings list to a string to store in CSV
df['feelings'] = df['feelings'].apply(lambda x: ', '.join(x))

# Save to CSV
df.to_csv(output_file, index=False)

print(f"Data exported to {output_file}")

ValueError: Trailing data

In [56]:
#process data for basic intent
whomadeyou_sample = df_train_raw.loc[df_train_raw['label'] == 'who_made_you'].sample(20, random_state=1)
whatcaniaskyou_sample = df_train_raw.loc[df_train_raw['label'] == 'what_can_i_ask_you'].sample(20, random_state=1)
whatsong_sample = df_train_raw.loc[df_train_raw['label'] == 'what_song'].sample(20, random_state=1)
whatareyourhob_sample = df_train_raw.loc[df_train_raw['label'] == 'what_are_your_hobbies'].sample(20, random_state=1)
doyouhavepets_sample = df_train_raw.loc[df_train_raw['label'] == 'do_you_have_pets'].sample(20, random_state=1)

whatcaniaskyou_sample.reset_index(drop=True, inplace=True)
whomadeyou_sample.reset_index(drop=True, inplace=True)
doyouhavepets_sample.reset_index(drop=True, inplace=True)
whatareyourhob_sample.reset_index(drop=True, inplace=True)
whatsong_sample.reset_index(drop=True, inplace=True)

merged_df = pd.concat([whomadeyou_sample, whatcaniaskyou_sample, whatareyourhob_sample, whatsong_sample, doyouhavepets_sample], ignore_index=True)
merged_df.loc[:,'label'] = 'smalltalk'
df_train_raw = pd.concat([df_train_raw, merged_df], ignore_index=True)
df_train_raw = df_train_raw.sample(frac=1, random_state=42).reset_index(drop=True)
df_train_raw

,query,label
0,i need more checkbooks delivered to me by mail,order_checks
1,id like a car rental between january 1st to th...,car_rental
2,order some new checks to replace the ones i ra...,order_checks
3,make a pto request for dates november 11 to bl...,pto_request
4,what's the purpose of existence,meaning_of_life
...,...,...
15095,how do i speed up the speech of this ai,change_speed
15096,how much gas is in my car,gas
15097,that’s not correct,no
15098,what do i need to make a vacation request with...,pto_request


In [64]:
basic_intents = ['greeting', 'goodbye', 'smalltalk']
df_train = df_train_raw.loc[df_train_raw['label'].isin(basic_intents)].reset_index(drop=True)
df_train.rename(columns={'query': 'text', 'label': 'intent'}, inplace=True)
df_train

,text,intent
0,what types of activities do you enjoy,smalltalk
1,cats or dogs,smalltalk
2,tell me what company programmed you,smalltalk
3,can you tell me about the pets you have,smalltalk
4,see ya later,goodbye
...,...,...
295,what questions do you respond to,smalltalk
296,ai how are you doing today,greeting
297,see you around,goodbye
298,peace out,goodbye


In [58]:
df_emotion = pd.read_csv('../dataset/goemotions_2.csv')
df_emotion.head()
# df_emotion.columns

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,We can hope,ee3o3ko,darkenseyreth,EdmontonOilers,t3_ag4r9j,t1_ee3mhad,1.547529e+09,62,False,0,...,0,0,1,0,0,0,0,0,0,0
1,Shhh don't give them the idea!,eebl3z7,BoinkBoinkEtAliae,MurderedByWords,t3_ah3o76,t1_eeb68lo,1.547777e+09,76,False,0,...,0,0,0,0,0,0,0,0,0,0
2,"Thank you so much, kind stranger. I really nee...",ed4fe9l,savageleaf,raisedbynarcissists,t3_abwh00,t1_ed4etbj,1.546482e+09,24,False,0,...,0,0,0,0,0,0,0,0,0,0
3,Ion know but it would be better for you to jus...,efavtdu,CADDiLLXC,darknet,t3_al4njw,t3_al4njw,1.548800e+09,62,False,0,...,0,0,0,0,0,0,0,0,0,1
4,I'm honestly surprised. We should have fallen ...,ee2imz2,CorporalThornberry,CollegeBasketball,t3_afxt6t,t1_ee22nyr,1.547497e+09,55,False,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
columns_keep = ['text', 'embarrassment', 'fear', 'grief', 'nervousness']
emotions = ['embarrassment', 'fear', 'grief', 'nervousness']

df_emotion = df_emotion[columns_keep]
df_emotion

# Create a boolean mask that is True if any of the specified columns equals 1
mask = df_emotion[emotions].eq(1).any(axis=1)

# Filter the DataFrame using the mask
result_emotion = df_emotion[mask]

# Print the result
result_emotion.reset_index(drop=True, inplace=True)
result_emotion

,text,embarrassment,fear,grief,nervousness
0,One of my worst nightmares. Ugh.,0,1,0,0
1,and everyone else is worried about themselves ...,0,0,0,1
2,Get it out of the way as quickly as possible. ...,0,1,0,0
3,group wasn't bad. Edit: Unless you're talking ...,1,0,0,0
4,This sounds like fainting. Get your blood pres...,1,0,0,1
...,...,...,...,...,...
2498,"Oh no, the horror. Look, a pause! Let's start ...",0,1,0,1
2499,You came to the wrong neighborhood bud lololol.,1,0,0,0
2500,I've never woken up to a fan around my throat.,1,0,0,0
2501,There are some amazingly cringey compilations ...,0,1,0,0


In [65]:
embarrassment_sample = result_emotion.loc[result_emotion['embarrassment'] == 1].sample(25, random_state=1)
fear_sample = result_emotion.loc[result_emotion['fear'] == 1].sample(25, random_state=1)
grief_sample = result_emotion.loc[result_emotion['grief'] == 1].sample(25, random_state=1)
nervousness_sample = result_emotion.loc[result_emotion['nervousness'] == 1].sample(25, random_state=1)

embarrassment_sample.reset_index(drop=True, inplace=True)
fear_sample.reset_index(drop=True, inplace=True)
grief_sample.reset_index(drop=True, inplace=True)
nervousness_sample.reset_index(drop=True, inplace=True)

emotion_merged = pd.concat([embarrassment_sample, fear_sample, grief_sample, nervousness_sample], ignore_index=True)
emotion_merged.loc[:,'intent'] = 'emotion'
emotion_merged.drop(columns=['embarrassment', 'fear', 'grief', 'nervousness'], inplace=True, axis=1)
df_train = pd.concat([df_train, emotion_merged], ignore_index=True)
df_train

,text,intent
0,what types of activities do you enjoy,smalltalk
1,cats or dogs,smalltalk
2,tell me what company programmed you,smalltalk
3,can you tell me about the pets you have,smalltalk
4,see ya later,goodbye
...,...,...
395,My knees shattered and my collarbone broke jus...,emotion
396,So looks like it was gang violence?,emotion
397,I ride our half-orc barbarian by clinging to h...,emotion
398,"Oh [NAME], oh no, no no no",emotion


In [73]:
df_suicide = pd.read_csv('../dataset/suicide_detection.csv')
df_suicide.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
df_suicide.drop(columns=['Unnamed: 0'], inplace=True)
df_suicide.rename(columns={'class': 'intent'}, inplace=True)

In [78]:
df_suicide = df_suicide.loc[df_suicide['intent'] == 'suicide', :]
df_suicide = df_suicide.sample(100, random_state=1).reset_index(drop=True)
df_suicide

,text,intent
0,"March 5, 2022That’s when my life will be over ...",suicide
1,Too many bad daysI'm not feeling up for the st...,suicide
2,Aftermath. Help?I don't know if this is the ri...,suicide
3,Ignored and neglectedI do so much 4 those aro...,suicide
4,I'll end this my way.Fuck living in this word....,suicide
...,...,...
95,I can almost feel freedomI'm not quite sure wh...,suicide
96,ventNovember is when I no longer have any obli...,suicide
97,I am existingSuicide is so simple with how it ...,suicide
98,I just want the pain to stop for goodI’ve been...,suicide


In [79]:
df_train = pd.concat([df_suicide, df_train], ignore_index=True)
df_train

,text,intent
0,"March 5, 2022That’s when my life will be over ...",suicide
1,Too many bad daysI'm not feeling up for the st...,suicide
2,Aftermath. Help?I don't know if this is the ri...,suicide
3,Ignored and neglectedI do so much 4 those aro...,suicide
4,I'll end this my way.Fuck living in this word....,suicide
...,...,...
495,My knees shattered and my collarbone broke jus...,emotion
496,So looks like it was gang violence?,emotion
497,I ride our half-orc barbarian by clinging to h...,emotion
498,"Oh [NAME], oh no, no no no",emotion
